<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parameter-settings" data-toc-modified-id="Parameter-settings-1">Parameter settings</a></span></li><li><span><a href="#Convert-the-temperature-.csv-file-to-NetCDF-file" data-toc-modified-id="Convert-the-temperature-.csv-file-to-NetCDF-file-2">Convert the temperature .csv file to NetCDF file</a></span></li><li><span><a href="#Generate-the-template-prior.nc-file" data-toc-modified-id="Generate-the-template-prior.nc-file-3">Generate the template prior.nc file</a></span></li><li><span><a href="#Generate-the-DART-variable-quantity-files" data-toc-modified-id="Generate-the-DART-variable-quantity-files-4">Generate the DART variable quantity files</a></span></li><li><span><a href="#Generate-the-DART-observation-file-from-NetCDF-observation-file" data-toc-modified-id="Generate-the-DART-observation-file-from-NetCDF-observation-file-5">Generate the DART observation file from NetCDF observation file</a></span><ul class="toc-item"><li><span><a href="#Prepare-the-convert_nc.f90-file" data-toc-modified-id="Prepare-the-convert_nc.f90-file-5.1">Prepare the convert_nc.f90 file</a></span></li><li><span><a href="#Compile-convert_nc.f90-file" data-toc-modified-id="Compile-convert_nc.f90-file-5.2">Compile convert_nc.f90 file</a></span></li><li><span><a href="#Generate-the-DART-observation-file-by-using-the-convert_nc.f90" data-toc-modified-id="Generate-the-DART-observation-file-by-using-the-convert_nc.f90-5.3">Generate the DART observation file by using the convert_nc.f90</a></span></li></ul></li></ul></div>

**This notebook is used for**
- generating the [DART](https://www.image.ucar.edu/DAReS/DART/Manhattan/documentation/index.html) observation file (obs_seq.out) from observation data in NetCDF file
- generating the template ```prior.nc``` file (or DART restart file) from the results of PFLOTRAN spin-up
- generating the DART variable quantities files

In [4]:
import numpy as np

# Parameter settings

In [5]:
# Directories
obs_kind_dir    = '../obs_kind/'
obs_type_dir    = '../obs_type/'
obs_convert_dir = '../obs_converter/'
utils_dir       = '../utils/'
data_dir        = '../data_files/'

# DART file names
def_obs_kind = obs_kind_dir+'DEFAULT_obs_kind_mod.f90'
obs_type     = obs_type_dir+'obs_def_pflotran_mod.f90'

# Utility file names
csv_to_nc        = utils_dir+'csv2nc.py'
prep_convert_nc  = utils_dir+'prepare_convert_nc.py'
compile_convert_nc = obs_convert_dir+'dart_seq_convert.csh'
convert_nc       = obs_convert_dir+'convert_nc'
to_dartqty       = utils_dir+'list2dartqty.py'
obs_type_qty_ind = utils_dir+'obs_type_qty_ind.txt'

# Data file names
obs_original = data_dir+'temperature.csv'
obs_nc       = data_dir+'obs_pflotran.nc'
obs_dart     = data_dir+'obs_seq_pflotran.out'
prior_nc     = data_dir+'prior_template_pflotran.nc'

# Specify PFLOTRAN variables used as observation and state vector/parameters in DART
obs_var_set  = ['TEMPERATURE']
para_set     = ['PERMEABILITY','POROSITY','THERMALCONDUCTIVITY']
state_set    = ['LIQUID_SATURATION','LIQUID_PRESSURE', 'TEST_VARIABLE', 'TEST_VARIABLEAAA']
pflotran_set = obs_var_set + para_set + state_set

# Convert the temperature .csv file to NetCDF file
Note that 
- in reality the original observation file format varies. Therefore, users are required to prepare their own .nc file;
- in your converted .nc file, it needs to contain temporal and spatial information, which are required by DART.

In [6]:
%%script bash -s "$csv_to_nc" "$obs_original" "$obs_nc"
python $1 $2 $3

(8641, 5)


# Generate the template prior.nc file

The template prior.nc file should include:
- the parameter (from ```parameter.h5``` -- input) or the state vector (from PFLOTRAN spin-up output)
- the predicted observations (from PFLOTRAN spin-up output)
- the time information (from PFLOTRAN spin-up output)
- the spatial inforamtion (from PFLOTRAN spin-up output)

In [ ]:
%%script bash
pflotran

# Generate the DART variable quantity files
It includes [DEFAULT_obs_kind_mod.f90](https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/modules/observations/DEFAULT_obs_kind_mod.html) and [obs_def_mod.f90](https://www.image.ucar.edu/DAReS/DART/Manhattan/observations/forward_operators/obs_def_mod.html) (file names can be changed). The paths of the two files are specified in [&preprocess_nml](https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/programs/preprocess/preprocess.html) in ```input.nml``` file. 

In [4]:
%%script bash -s "$to_dartqty" "$obs_type" "$def_obs_kind" "$pflotran_set"
python $1 $2 $3 $4

The added DART variable quantities names and indices are ...
['QTY_PFLOTRAN_TEST_VARIABLEAAA', 'QTY_PFLOTRAN_TEST_VARIABLE']
[373, 374]


# Generate the DART observation file from NetCDF observation file
Generate DART observation file requires the preparations of both .nc observation file and DART variable quantity files. An overview of DART observation file can be found at [link](https://www.image.ucar.edu/DAReS/DART/DART2_Observations.html#obs_seq_overview). Basically, we first creat the fortran code ```convert_nc.f90``` for generating the DART observation, and compile the fortran. Then, run the executable ```conver_nc``` to generate DART observation file ```obs_dart```

## Prepare the convert_nc.f90 file

In [16]:
%%script bash -s "$prep_convert_nc" "$obs_nc"
python $1 $2

## Compile convert_nc.f90 file

In [17]:
%%script bash -s "$obs_convert_dir" "$compile_convert_nc"
cd $1
csh $2

---------------------------------------------------------------
Removing *.o *.mod files


---------------------------------------------------------------
NetCDF converters build number 1 is preprocess
 Makefile is ready.
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/types_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/utilities_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/programs/preprocess/preprocess.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/time_manager_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/null_mpi_utilities_mod.f90
gfortran preprocess.o utilities_mod.

.........................................

## Generate the DART observation file by using the convert_nc.f90

In [19]:
%%script bash -s "$obs_convert_dir" "$convert_nc"
cd $1
$2


 --------------------------------------
 Starting ... at YYYY MM DD HH MM SS = 
                 2019  9 12 17 37 16
 Program convert_nc
 --------------------------------------

  set_nml_output Echo NML values to log file only
  write_obs_seq  opening formatted observation sequence file "../data_files/obs_seq.pflotran"
  write_obs_seq  closed observation sequence file "../data_files/obs_seq.pflotran"

 --------------------------------------
 Finished ... at YYYY MM DD HH MM SS = 
                 2019  9 12 17 37 18
 --------------------------------------

